# Sentiment Analysis

### 1) Classifier per Category

In [1]:
## Scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

## Librerias para graficación
import matplotlib.pyplot as plt
import seaborn as sns

# NLTK es una librería particular para PLN. Tiene muchas funcionalidades entre ellas stemming y lista de palabras de parada.
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import os
import csv

stemmer = nltk.stem.SnowballStemmer('english') # Vamos a utlizar el Snowball Stemmer para realizar stemming (nos permite llevar las palabras a una forma estandar).
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#Preprocessing of words
def processing_text(texto):
    processed_feature = re.sub(r'\W', ' ', str(texto))
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)

    return processed_feature

In [3]:
# Define function to read and preprocess the reviews
def read_and_preprocess_reviews(folder_path):
    
    dictionary_training = {}
    dictionary_test = {}

    #Define stopwords
    stopwords_english = set(stopwords.words('english'))

    for category in os.listdir(folder_path):
        category_path = os.path.join(folder_path, category) #Category path
        if os.path.isdir(category_path):
            for sentiment in ["positive","negative"]:
                sentiment_path = os.path.join(category_path, sentiment + '.review') #Sentiment path
                if os.path.isfile(sentiment_path):
                    with open(sentiment_path, 'r',encoding='utf-8') as file:
                        review_text = file.read()
                        # Preprocess input data
                        preprocess = processing_text(review_text)
                        words = nltk.word_tokenize(preprocess)
                        string = ""
                        train_reviews = []
                        train_labels = []
                        
                        for word in words:
                            if word == "positive" or word == "negative":
                                train_labels.append(word)
                                train_reviews.append(string)
                                string = ""
                            else:
                                string += word + " "
                        
        # list of reviews per category with their respective class(posituive or negative)     
        dictionary_training[category] = (train_reviews,train_labels)

        test_reviews = []
        test_label = []
        #Test set
        sentiment_test_path = os.path.join(category_path, 'unlabeled.review') #Sentiment path test
        if os.path.isfile(sentiment_test_path):
            with open(sentiment_test_path, 'r',encoding='utf-8') as file:
                review_text = file.read()
                # Preprocess and remove stopwords
                preprocess = processing_text(review_text)
                words = nltk.word_tokenize(preprocess)
                for word in words:
                    if word == "positive" or word == "negative":
                        test_label.append(word)
                        test_reviews.append(string)
                        string = ""
                    else:
                        string += word + " "
                        
        dictionary_test[category] = (test_reviews,test_label)

    return dictionary_training,dictionary_test
                

data_folder = './Multi Domain Sentiment/processed_acl'
reviews_training_set = read_and_preprocess_reviews(data_folder)[0]
unlabeled_test_text = read_and_preprocess_reviews(data_folder)[1]

In [4]:
print(reviews_training_set["kitchen"][0][0])
#print(len(unlabeled_test_text))

right_after guess dog well fill to_work it_seems keep other_small work_ num working_very care barks cairn small_dog time and fairly sure_why is_supposed emits and_barks just_looks either_does such_a for_my either away emits and such sure it_either my they re_getting again it to_care the_spray care_if does_not a_persistent getting_sprayed just my_other sprays_as we re_not looks sprays if_they re are_just is_not my_yappy too_stubborn doesn t_work looks_away when dog_who barker barking they re we_fill fill_it not_sure work sprayed_i long it_does also seems also_doesn small cairn_from we sprayed again it_also yappy persistent_barker where not_such persistent why terriers isn t_working long it who if just_too very num _of time and_we re from_where right work_well he very_long it well_for supposed_to too when_we terriers_are we re after_but keep_my num other getting after stubborn yappy_cairn not the_time and as_many stubborn_to supposed where_the well_right fairly_well doesn work_fairly man

In [5]:
#Bag of words
vectorizer = CountVectorizer(max_features=2500, stop_words=stopwords.words('english'))
# Vocabulario y tambien transforme nuestro texto
for categories in reviews_training_set:
    
    reviews = reviews_training_set[categories][0]
    labels = reviews_training_set[categories][1]
    
    unlabeled_reviews = unlabeled_test_text[categories][0]
    unlabeled_labels = unlabeled_test_text[categories][1]

    texto_features = vectorizer.fit_transform(reviews).toarray()

    #Entrenar el modelo naive bayes
    nb = MultinomialNB()
    nb.fit(texto_features, labels)

    #Evaluar que tan buneo es el modelo
    predictions = nb.predict(unlabeled_reviews)

    # Ahora calculamos el score de accuracy enviando las predicciónes y los valores reales de polaridad.
    print(accuracy_score(unlabeled_labels, predictions))

ValueError: Expected 2D array, got 1D array instead:
array=['avid your horrible_book wasted use_it the_entire money i_lit i_read lit i_would relationship read a_ num reader_and reader suffering fire_one i_had year_old gotten horrible lit_this world don my one_star headache_the this_book mom was_horrible friend book_horrible star_i back avid_reader than_one life copy rate_it rate my_mom man book_was half on_fire and_then reading_this so lower i_could num _year than time half_of time_spent then book and_picked possible spent old_man up_after one horrible_if one_less part was entire less_copy to_rate my_life about_the your_money an_avid if the_relationship use a_headache fire lower_than reading a_friend picked purposes then_got waste_your after_my friend_i old man_and and_i world don t_waste book_on part_about copy_in book_back book_wasted have_i time_and the_world don better if_it star got mom_had read_half waste after about could_use had_gotten was_possible year it_lower relationship_th